In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import re
import time
import pandas as pd

filename = "data"
search_queries = [
    "https://www.google.com/maps/search/%D1%88%D0%BA%D0%BE%D0%BB%D0%B0+%D0%B2+%D1%82%D0%B0%D1%88%D0%BA%D0%B5%D0%BD%D1%82%D0%B5/@41.3113397,69.251939,15z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/maktab+%D0%B2+%D0%A2%D0%B0%D1%88%D0%BA%D0%B5%D0%BD%D1%82/@41.3064006,69.2091373,11z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B0%D1%8F+%D1%88%D0%BA%D0%BE%D0%BB%D0%B0+%D0%B2+%D0%A2%D0%B0%D1%88%D0%BA%D0%B5%D0%BD%D1%82/@41.334712,69.2505282,11.86z?entry=ttu",
    "https://www.google.com/maps/search/%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F+%D1%88%D0%BA%D0%BE%D0%BB%D0%B0+%D0%B2+%D0%A2%D0%B0%D1%88%D0%BA%D0%B5%D0%BD%D1%82/@41.334641,69.2501506,11z/data=!3m1!4b1?entry=ttu"
    "https://www.google.com/maps/search/%D0%BF%D0%B0%D1%80%D0%BA%D0%B8+/@41.3113172,69.2210391,13z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/korzinka/@41.3112584,69.221039,13z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/havas/@41.3112438,69.221039,13z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/baraka/@41.3112291,69.221039,13z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/mall/@41.3112144,69.2210389,13z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/%D0%B5%D0%B4%D0%B0/@41.3302784,69.2584448,12z?entry=ttu",
    "https://www.google.com/maps/search/%D0%BF%D0%B0%D1%80%D0%BA/@41.3301609,69.2584443,12z/data=!3m1!4b1?entry=ttu",
    "https://www.google.com/maps/search/%D0%B1%D0%B0%D0%B7%D0%B0%D1%80/@41.3301021,69.2584441,12z/data=!3m1!4b1?entry=ttu"
]

record = []
e = []
le = 0

def Selenium_extractor(query):
    link = f"{query}&hl=ru"
    browser = webdriver.Chrome()
    browser.get(link)
    time.sleep(5)
    action = ActionChains(browser)
    a = browser.find_elements(By.CLASS_NAME, "hfpxzc")
    
    while len(a) < 1000:
        print(len(a))
        var = len(a)
        scroll_origin = ScrollOrigin.from_element(a[0])
        action.scroll_from_origin(scroll_origin, 0, 20000).perform()
        time.sleep(3)
        a = browser.find_elements(By.CLASS_NAME, "hfpxzc")

        if len(a) == var:
            le+=1
            if le > 2:
                break
        else:
            le = 0    
    
    print(f"le {le}")
    print(f"a {len(a)}")


    browser.implicitly_wait(10)
    html_content = browser.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    div_elements = soup.find_all('div', class_=re.compile(r'^Nv2PK '))

    for div_element in div_elements:
        name = div_element.find('a', class_='hfpxzc')['aria-label']
        category_parent_divs = div_element.find_all('div', class_='W4Efsd')
        if len(category_parent_divs) >= 2:
            category_span =category_parent_divs[1].select('span > span')
            category = category_span[0].text.strip()
        else:
            category = ''
        href = div_element.find('a', class_='hfpxzc')['href']
        latitude = re.search(r'!3d(.*?)!4d(.*?)!', href).group(1)
        longitude = re.search(r'!3d(.*?)!4d(.*?)!', href).group(2)
        if name not in record:
            record.append((name,category,latitude,longitude))
 

for query in search_queries:
    Selenium_extractor(query)
    
print(len(record))

10
19
21
27
33
39
41
47
53
59
61
67
73
79
81
87
93
99
101
107
113
119
121
121
121
le 3
a 121
10
19
20
29
38
40
49
58
60
69
78
80
89
98
100
109
118
120
120
120
le 3
a 120
10
19
21
30
39
41
50
59
61
70
79
81
81
81
le 3
a 81
10
19
20
23
23
23
le 3
a 23
10
19
20
29
38
40
49
58
60
69
78
80
89
98
100
109
118
118
118
le 3
a 118
7
13
19
20
26
32
38
40
46
52
58
60
66
72
78
80
86
92
98
100
106
112
118
120
120
120
le 3
a 120
9
18
20
29
38
40
49
58
60
69
78
80
82
82
82
le 3
a 82
7
13
19
20
26
32
38
40
46
52
58
60
66
72
78
80
86
92
98
100
106
112
118
120
120
120
le 3
a 120
7
13
19
20
26
32
38
40
46
52
58
60
66
72
78
80
86
92
98
100
106
112
118
120
120
120
le 3
a 120
7
13
19
20
26
32
38
40
46
52
58
60
66
72
78
80
86
92
98
100
106
112
118
120
120
120
le 3
a 120
10
19
20
29
38
40
49
58
60
69
78
80
89
98
100
109
118
120
120
120
le 3
a 120
1145


In [2]:
df = pd.DataFrame(record, columns=['name', 'category', 'lat', 'long'])
file_name = '..\Data\Map_Data.xlsx'
df.to_excel(file_name, index=False)